# Utility code for identifying primes and prime covered by $\mathcal{G}(29^\#)$
This is very specific code for identifying arrays of consecutive primes within an interval.
The interval is defined by its starting point $x_0$ and its length 2*<i>blocklength</i>.  Since our algorithm only
searches over odd numbers, the parameter <i>blocklength</i> effects a search from $x_0$ to $x_0 + 2\cdot {\rm blocklength}$.

The search is performed by the function <i>pblock11()</i>.  This is Eratosthenes sieve with a couple of accelerations.  We search
only over odd numbers, each odd number is represented by a single bit in a boolean array, and we initialize the array with the
pattern from ${\mathcal G}(11^\#)$.  So the iterations start from $p=13$.

Our search is conducted by eliminating multiples of the primes in the array <i>smallprimes[]</i>.  So the range of our search
wanes at <i>smallprimes[-1]**2</i>, the square of the largest prime in this array.

In [1]:
import numpy as np
import array
import gc
import psutil
import sys
import os

In [2]:
# block to check the available system memory
gc.collect()
memory = psutil.virtual_memory()
available_memory = memory.available
del memory
print(f"Available memory: {available_memory / (1024 ** 2):.2f} MB")

Available memory: 2645.88 MB


In [3]:
# create the binary pattern for 11-rough numbers - note - we only look at the odd numbers
try:
    G11 = np.load("G11uint.npy")
except FileNotFoundError:
    print("File G11uint.npy not found!")
    sys.exit()
    
rough11 = np.zeros(1155, dtype=bool)
i=0
j=0
while (i < 1155):
    rough11[i]=True
    i += int(G11[j]/2)
    j += 1

rough11[-100:]

array([ True,  True, False,  True,  True, False, False, False, False,
        True,  True, False, False,  True, False,  True,  True, False,
        True, False, False,  True, False, False,  True,  True, False,
       False, False, False,  True,  True, False, False,  True, False,
        True, False, False, False, False, False, False,  True, False,
        True,  True, False,  True,  True, False,  True, False, False,
       False,  True, False, False,  True, False,  True, False, False,
        True,  True, False,  True, False, False,  True,  True, False,
       False,  True, False, False,  True, False,  True,  True, False,
        True, False, False,  True,  True, False, False,  True, False,
        True,  True, False,  True, False, False, False, False, False,
        True])

In [4]:
# identify the primes among odd numbers in a block of size blocklength starting at x0
# this version repeats the pattern from G(11#) to accelerate execution.
# 
def pblock11(x0, blocklength):
    maxp = np.floor(np.sqrt(x0+2*blocklength))
    if (maxp > smallprimes[-1]):
        print(f"ERROR: need prime {maxp} beyond available list {smallprimes[-1]}")
        return

    if (x0 <= maxp):  # the starting point is within the range of primes needed in the calculation
        xx0 = maxp
    else:
        xx0 = x0

    if ((xx0 % 2) == 0):  # our starting point xx0 must be odd
        xx0 = xx0+1

    # value associated to index i is xx0+2i
    # initialize the array with the pattern from G(11#)
    # 
    res0 = int(xx0 % 2310)
    i = int((res0 - 1)/2)  # map value to index in boolean block over odd numbers
    ncopies = int(np.ceil(blocklength / 1155))
    adj_blocklen = ncopies * 1155

    # if needed, adjust the maximum prime that we need to check
    maxp1 = np.floor(np.sqrt(xx0 + 2*adj_blocklen))
    if maxp1 > maxp:
        print(f"Adjusted maxp {maxp} {blocklength} to {maxp1} {adj_blocklen}")
        maxp = maxp1
    
    if (i == 0):
        is_prime = np.tile(rough11, ncopies)
    else:
        shifted_rough11 = np.concatenate((rough11[i:],rough11[0:i]))
        is_prime = np.tile(shifted_rough11, ncopies)

#    is_prime = np.ones(blocklength,dtype=bool)
    print(f"res0 {res0} i {i} x0 {xx0} block {len(is_prime)} pbound {maxp}")

    i=4 # index for smallprimes, start at p=13
    j=0 # index for testlength
    p = smallprimes[i]
    if (p != 13):
        print(f"i {i} prime {p} should be 13")
        return
        
    while (p < maxp):
        if (p < 3000000):
            print(f"checking {p} of {maxp}", end='\r')
        elif ( (i % 1024) == 0):
            print(f"{i} checking {p} of {maxp}", end='\r')
            
        res0 = xx0 % p  # starting residue
        delta0 = int(p - res0)
        if (res0 == 0):
            j = 0
        elif ((delta0 % 2) ==0):
            j = int(delta0 /2)
        else:
            j = int((p+delta0)/2)
        while (j < adj_blocklen):
            is_prime[j] = False
            j += p
        i += 1
        p = smallprimes[i]

    pdexes = np.array(np.where(is_prime))
    pdexes.shape

    newprimes = (2*pdexes[0]+xx0)
    print()
    print(f"at {i} {len(newprimes)} primes from {newprimes[0]} to {newprimes[-1]}")
    return newprimes

In [5]:
# find our starting point - 
if os.path.exists('primesE9.npy'):
    primesE9 = np.load('primesE9.npy')
    print(f"primesE9 {len(primesE9)} primes from {primesE9[0]} to {primesE9[-1]}")
    smallprimes = primesE9
elif os.path.exists('primes23.npy'):
    primes23 = np.load('primes23.npy')
    print(f"primes23 {len(primes23)} primes from {primes23[0]} to {primes23[-1]}")
    smallprimes = np.concatenate(([3,5,7,11,13,19,23],primes23))
    # Create primesE9
    x0 = smallprimes[-1]
    ext_length = 400000000
    prime_extension = pblock11(x0,ext_length)
    # store primesE9 to file for future use
    smallprimes = np.concatenate(([3,5,7,11,13,17,19,23],primes23[0:],prime_extension[1:]))
    np.save('primesE9.npy', smallprimes)
else:
    print("ERROR:  we need primes23.npy to start")
    sys.exit()

primesE9 51961553 primes from 3 to 1023094327


## Example usage of pblock11()
We set a starting point $x_0$ for the search.  $x_0$ should be a large odd integer.
The requested interval over which we search begins at $x_0$ and runs 2*<i>blocklength</i>.  We invoke <i>pblock11(x0, blocklength)</i>
with the parameter <i>blocklength</i>, and since we only search over odd numbers

In [6]:
# test cell
# we want to focus on intervals of survival, so start at a prime-square and allow for the square falling in a large gap
x0 = smallprimes[-100000]**2 - 1000  
testlength = 900000000
ptrial = pblock11(x0,testlength)

res0 2151 i 1075 x0 1042480837758600081 block 900000255 pbound 1021019509.0
51861504 checking 1021018513 of 1021019509.0
43387768 primes from 1042480837758600169 to 1042480839558600569


In [11]:
# visually check the result...
print(ptrial[0:30])
print()
print(ptrial[-30:])

[1042480837758600169 1042480837758600173 1042480837758600179
 1042480837758600209 1042480837758600211 1042480837758600223
 1042480837758600287 1042480837758600319 1042480837758600371
 1042480837758600403 1042480837758600463 1042480837758600473
 1042480837758600503 1042480837758600563 1042480837758600571
 1042480837758600593 1042480837758600629 1042480837758600671
 1042480837758600677 1042480837758600689 1042480837758600809
 1042480837758600841 1042480837758600923 1042480837758600949
 1042480837758601049 1042480837758601061 1042480837758601081
 1042480837758601121 1042480837758601237 1042480837758601301]

[1042480839558599041 1042480839558599053 1042480839558599179
 1042480839558599209 1042480839558599249 1042480839558599393
 1042480839558599401 1042480839558599437 1042480839558599501
 1042480839558599557 1042480839558599591 1042480839558599671
 1042480839558599699 1042480839558599743 1042480839558599893
 1042480839558599921 1042480839558599971 1042480839558599977
 1042480839558600017 1

## File naming convention
To organize the files of the saved arrays of primes we use the following naming convention.

Each file begins with the prefix <i>primeblock</i> followed by a reverse scientific notation, the <i>E</i> value (the exponent)
followed by an underscore and then the first three digits of the coefficient, for the largest prime in the array.

In [13]:
# SAVE this data for use in other modules
# we comment out these lines to prevent accidental clobbering of the existing files
# np.save('primeblockEXX.npy',ptrial)